In [109]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools
from IPython.display import display
import os
import csv
from textblob import TextBlob

In [12]:
#Search term to be used on twitter scrape
search = "@McDonalds"

#how to scrape
scraped_tweets= sntwitter.TwitterSearchScraper(search).get_items()
sliced_scraped_tweets = itertools.islice(scraped_tweets,10)

#create df of all the new data
df = pd.DataFrame(sliced_scraped_tweets)[['date', 'content']]

In [13]:
df

,date,content
0,2021-07-13 13:17:19+00:00,@HoldenPro @ProWilldy @McDonalds Mmmmmm caffei...
1,2021-07-13 13:17:14+00:00,From embarking on long journeys ✈️ and in need...
2,2021-07-13 13:17:08+00:00,@CheckersRallys @KrisAndersonTV @BurgerKing @M...
3,2021-07-13 13:16:55+00:00,I love McDonalds French fries and fish sandwic...
4,2021-07-13 13:15:56+00:00,@KrisAndersonTV @BurgerKing @McDonalds @Chickf...
5,2021-07-13 13:15:03+00:00,@crockettonair @foodbeast @McDonalds @Chickfil...
6,2021-07-13 13:14:31+00:00,#faistonbeurre #fashions #hype #hope #undergro...
7,2021-07-13 13:13:55+00:00,@kwilesjrphoto @KrisAndersonTV @BurgerKing @Mc...
8,2021-07-13 13:13:17+00:00,I live at McDonald’s and by that I mean I live...
9,2021-07-13 13:12:26+00:00,@Bron1954 @drunkvegans @BudderedUp @Aivan10100...


In [14]:
#create array that will store the names of all the cities
city_array = []

#import dataframe containing populations of cities in california
california_cities_df = pd.read_csv('cal_populations_city.csv')


In [15]:
california_cities_df['pop_april_2010'].count()

455

In [16]:
california_cities_df.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
0,San Bernardino,Adelanto,1970,2164.0,8517,18130,31765
1,Los Angeles,Agoura Hills,1982,NaN,20390,20537,20330
2,Alameda,Alameda,1854,63852.0,76459,72259,73812
3,Alameda,Albany,1908,15130.0,16327,16444,18539
4,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089


In [71]:
#only want to look at cities bigger than 75k to ensure there is enough data
bigger_ca_cities_df = california_cities_df[california_cities_df['pop_april_2010']>75000] 

#rslt_df = dataframe[dataframe['Percentage'] > 70] 

In [73]:
bigger_ca_cities_df.count()

#100 cities meet the criteria

County                 100
City                   100
Incorportation_date    100
pop_april_1980          95
pop_april_1990         100
pop_april_2000         100
pop_april_2010         100
dtype: int64

In [74]:
#df of bigger cities in CA
bigger_ca_cities_df.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
4,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089
7,Orange,Anaheim,1876,219494.0,266406,328014,336265
10,Contra Costa,Antioch,1872,42683.0,62195,90532,102372
24,Kern,Bakersfield,1873,105611.0,174820,246889,347483
25,Los Angeles,Baldwin Park,1956,50554.0,69330,75837,75390


In [75]:
city_array = bigger_ca_cities_df['City']

In [76]:
#import dataframe containing city coordinates
city_latlng = pd.read_csv('cal_cities_lat_long.csv')

In [77]:
#rename columns
city_latlng = city_latlng.rename(columns={'Name': 'City', 'Latitude': 'lat', 'Longitude': 'lng'})

city_latlng.head()

,City,lat,lng
0,Adelanto,34.582769,-117.409214
1,Agoura Hills,34.153339,-118.761675
2,Alameda,37.765206,-122.241636
3,Albany,37.886869,-122.297747
4,Alhambra,34.095286,-118.127014


In [78]:
#merge columns together, so have city populations, and long/lat
city_df_merged = bigger_ca_cities_df.merge(city_latlng, how='inner', on='City')

In [79]:
#
city_df_merged.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010,lat,lng
0,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089,34.095286,-118.127014
1,Orange,Anaheim,1876,219494.0,266406,328014,336265,33.835292,-117.914503
2,Contra Costa,Antioch,1872,42683.0,62195,90532,102372,38.004922,-121.805789
3,Kern,Bakersfield,1873,105611.0,174820,246889,347483,35.373292,-119.018711
4,Los Angeles,Baldwin Park,1956,50554.0,69330,75837,75390,34.085286,-117.960897


In [80]:
final_city_df = city_df_merged[['City', 'pop_april_2010', 'lat', 'lng']]

In [81]:
final_city_df.count()

City              100
pop_april_2010    100
lat               100
lng               100
dtype: int64

In [82]:
final_city_df = final_city_df.rename(columns={'City':'city'})

In [83]:
final_city_df

,city,pop_april_2010,lat,lng
0,Alhambra,83089,34.095286,-118.127014
1,Anaheim,336265,33.835292,-117.914503
2,Antioch,102372,38.004922,-121.805789
3,Bakersfield,347483,35.373292,-119.018711
4,Baldwin Park,75390,34.085286,-117.960897
...,...,...,...,...
95,Visalia,124442,36.330228,-119.292058
96,Vista,93834,33.200036,-117.242536
97,West Covina,106098,34.068622,-117.938953
98,Westminster,89701,33.751342,-117.993992


In [84]:
final_city_len = final_city_df['lat'].count()

In [87]:
##tests
#lat = "37.765206"
#lng = "-122.241636"


df_array = []
i = 0
#cycle through every 
#while i < final_city_df['lat'].count():
    
#testing
while i < final_city_len:
    #creates a string for twitter scrapper to read and use as location
    loc = f"{str(final_city_df.loc[i, 'lat'])}, {str(final_city_df.loc[i, 'lng'])}, 10km"
    
    df_test = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    '@McDonalds geocode:"{}"'.format(loc)).get_items(), 5000))[['date', 'content', 'user']]
    
    df_test['user_location'] = df_test['user'].apply(lambda x: x['location'])
    df_test['lat'] = final_city_df.loc[i, 'lat']
    df_test['lng'] = final_city_df.loc[i, 'lng']
    
    df_array.append(df_test)
    i = i +1

In [88]:
for i in df_array:
    display(i)

,date,content,user,user_location,lat,lng
0,2021-07-13 08:39:59+00:00,@McDonalds West San Fernando Valley #Entertain...,"{'username': 'peyo_dupont', 'id': 784296892427...","Los Angeles, CA",34.095286,-118.127014
1,2021-07-13 05:45:14+00:00,I quit smoking weed two years ago and I've lov...,"{'username': 'TurdThePurd', 'id': 274661842, '...","Los Angeles, CA",34.095286,-118.127014
2,2021-07-13 04:24:41+00:00,"@dims_son @McDonalds Вау, вот это цены","{'username': 'BonadventureP', 'id': 1721301864...",Los Angeles,34.095286,-118.127014
3,2021-07-13 01:11:36+00:00,"@Jc_BeMine @iwubbchipotle @McDonalds trust me,...","{'username': 'missremiashten', 'id': 105401647...","Los Angeles, CA",34.095286,-118.127014
4,2021-07-13 00:48:44+00:00,#ad y’all already know how excited i am about ...,"{'username': 'missremiashten', 'id': 105401647...","Los Angeles, CA",34.095286,-118.127014
...,...,...,...,...,...,...
110,2021-07-05 17:44:37+00:00,@CynthiaaThugg @McDonalds Kickback is canceled,"{'username': 'Turis_20', 'id': 298350016, 'dis...","Los Angeles, CA",34.095286,-118.127014
111,2021-07-05 17:30:53+00:00,@2GGstrides Is it @McDonalds,"{'username': 'XenosMccloud', 'id': 1099699633,...","Los Angeles, CA",34.095286,-118.127014
112,2021-07-05 15:53:57+00:00,came to @McDonalds and they don’t have ketchup...,"{'username': 'CynthiaaThugg', 'id': 156929965,...","Los Angeles, CA",34.095286,-118.127014
113,2021-07-02 03:06:49+00:00,"I'm at @McDonalds in Glendale, CA https://t.co...","{'username': 'EliMendez23', 'id': 113112785, '...","Mid-City LA, CA",34.095286,-118.127014


,date,content,user,user_location,lat,lng
0,2021-07-12 22:01:01+00:00,Do @McDonalds stop automatically giving out ke...,"{'username': 'litodragonx', 'id': 132120417501...","Anaheim, CA",33.835292,-117.914503
1,2021-07-11 23:11:30+00:00,Excited to watch two great teams tonight in th...,"{'username': 'NHLDucksGaming', 'id': 108599152...","Anaheim, CA",33.835292,-117.914503
2,2021-07-11 20:03:24+00:00,@chris_hurney @BizNasty2point0 @ryanwhitney6 @...,"{'username': 'JasoninYLCA', 'id': 20285277, 'd...","Yorba Linda, CA",33.835292,-117.914503
3,2021-07-11 17:51:26+00:00,When you order ahead at @McDonalds but still ...,"{'username': 'tracyejacobs', 'id': 82669192, '...","California, USA",33.835292,-117.914503
4,2021-07-10 19:29:00+00:00,@RealBrittBaker @AEW @AEWonTNT @tntdrama @McDo...,"{'username': 'ThisIsKhristian', 'id': 19365762...",Anaheim C.A,33.835292,-117.914503
...,...,...,...,...,...,...
1187,2010-12-20 02:59:06+00:00,Now that I have @carlsjr folllowing me its tim...,"{'username': 'Steviecouture', 'id': 50665806, ...","Hollywood, Los Angeles",33.835292,-117.914503
1188,2010-11-24 17:53:58+00:00,Holiday pies @mcdonalds — photo at McDonald's ...,"{'username': 'mainerinrancho', 'id': 31158757,...","Highlands Ranch, CO",33.835292,-117.914503
1189,2010-11-23 22:32:07+00:00,"@mcdonalds My All Time Favorite for Years ""Big...","{'username': 'cameronslick', 'id': 37276951, '...","iPhone: 33.728836,-117.900894",33.835292,-117.914503
1190,2010-09-11 21:04:49+00:00,My @mcdonalds mcflurry was mixed by hand! New ...,"{'username': 'targeteer2k', 'id': 16525097, 'd...","Orange County, CA, USA",33.835292,-117.914503


,date,content,user,user_location,lat,lng
0,2021-04-26 07:02:33+00:00,.@McDonalds Figure out a way to install update...,"{'username': 'TV4Fun', 'id': 37005151, 'displa...","California, USA",38.004922,-121.805789
1,2020-10-25 22:39:17+00:00,@McDonalds Bring back the Mc DLT! https://t.co...,"{'username': 'PhilSmith34', 'id': 23177102, 'd...","Oakland, CA",38.004922,-121.805789
2,2020-10-08 01:00:10+00:00,@McDonalds I ordered the j balvin meal. The lo...,"{'username': 'JujusFish', 'id': 250022160, 'di...",Bay Area,38.004922,-121.805789
3,2020-09-17 21:12:13+00:00,Yo @McDonalds.... it’s on sight out here in th...,"{'username': 'MixedbyJayTee', 'id': 467909228,...","Bay Area, CA",38.004922,-121.805789
4,2020-03-04 21:11:58+00:00,@McDonalds I just wanna let y’all know I love ...,"{'username': 'FlyboIsaiah', 'id': 604203530, '...","Antioch, CA",38.004922,-121.805789
5,2020-02-09 23:17:12+00:00,@georgegotjokes @McDonalds McDonald’s coke is ...,"{'username': 'blvckstvr_', 'id': 24105393, 'di...",Japan,38.004922,-121.805789
6,2019-12-05 04:01:15+00:00,Why does @McDonalds give you an order number t...,"{'username': 'JujusFish', 'id': 250022160, 'di...",Bay Area,38.004922,-121.805789
7,2019-12-04 17:47:37+00:00,@aartipaarti @_troyjohnson @ChefDPhillips @Foo...,"{'username': 'Speakerman72', 'id': 2481258032,...",Antioch,38.004922,-121.805789
8,2019-06-26 22:48:58+00:00,Good #Sustainability news @McDonaldsCanada @Mc...,"{'username': 'andytoronto', 'id': 79201672, 'd...","San Francisco, California",38.004922,-121.805789
9,2019-06-12 18:09:27+00:00,@McDonalds Y’all need to bring donut sticks ba...,"{'username': 'Its_bitesize', 'id': 356259481, ...","Bay Area, CA",38.004922,-121.805789


,date,content,user,user_location,lat,lng
0,2021-07-09 23:23:13+00:00,Thank you @McDonalds for doing the right thing...,"{'username': 'Lumpy7255', 'id': 73656515588712...","Bakersfield, CA",35.373292,-119.018711
1,2021-07-09 15:03:43+00:00,"I'm at @McDonalds in Bakersfield, CA https://t...","{'username': 'HawaiianBound', 'id': 19970360, ...","Burbank, CA",35.373292,-119.018711
2,2021-07-06 21:47:23+00:00,@TCrimsonrose This had to be some @McDonalds s...,"{'username': 'GhettoElegant', 'id': 3265743914...","Bakersfield, CA",35.373292,-119.018711
3,2021-07-06 01:26:08+00:00,@McDonalds i love you but please run me my mon...,"{'username': 'Honeybunparis', 'id': 3249364574...",sad girls club,35.373292,-119.018711
4,2021-07-04 21:49:06+00:00,@McDonalds count your fucken days,"{'username': 'freegreedobitch', 'id': 10847277...","California, USA",35.373292,-119.018711
...,...,...,...,...,...,...
213,2011-02-09 07:59:39+00:00,I had the Angus Deluxe Burger @McDonalds today...,"{'username': 'Kevinmartinelli', 'id': 14329633...","Bakersfield, California",35.373292,-119.018711
214,2011-01-31 20:03:02+00:00,@McD_Chicago the new maple & fruit oatmeal is ...,"{'username': 'Kevinmartinelli', 'id': 14329633...","Bakersfield, California",35.373292,-119.018711
215,2011-01-31 07:30:40+00:00,First thing in the Morning it's @McDonalds for...,"{'username': 'Kevinmartinelli', 'id': 14329633...","Bakersfield, California",35.373292,-119.018711
216,2011-01-26 18:49:57+00:00,@McDonalds I just had the Fruit & Maple Oatmea...,"{'username': 'Kevinmartinelli', 'id': 14329633...","Bakersfield, California",35.373292,-119.018711


,date,content,user,user_location,lat,lng
0,2021-07-08 01:31:58+00:00,@McDonalds I'll download your app after you do...,"{'username': 'Gilbert76679457', 'id': 90954154...","La Puente, CA",34.085286,-117.960897
1,2021-07-06 07:04:21+00:00,@McDonalds I see something I would never put i...,"{'username': 'NicoleSciarra', 'id': 40206475, ...",CaLiFoRnIa/OrEgOn 626/541,34.085286,-117.960897
2,2021-07-02 07:03:45+00:00,I’ve been seeing a lot of people talk about ho...,"{'username': 'admayral_me', 'id': 474295148, '...",,34.085286,-117.960897
3,2021-05-27 13:39:53+00:00,"@McDonalds Yes. Yes, I am. Don't care.","{'username': 'Jenn8590', 'id': 1486275330, 'di...",Californ -I- A,34.085286,-117.960897
4,2021-05-26 17:07:50+00:00,"Well that’s massively disappointing, @McDonald...","{'username': 'purple_shield', 'id': 1272129245...",United States,34.085286,-117.960897
...,...,...,...,...,...,...
837,2010-12-10 17:13:58+00:00,"I'm @mcdonalds (7503 Garvey Ave, Rosemead) ht...","{'username': 'cat1152114', 'id': 30341217, 'di...","ÜT: 34.129889,-118.069719",34.085286,-117.960897
838,2010-11-24 05:57:44+00:00,"I'm @mcdonalds (6910 Rosemead Blvd, San Gabrie...","{'username': 'cat1152114', 'id': 30341217, 'di...","ÜT: 34.129889,-118.069719",34.085286,-117.960897
839,2010-11-21 03:39:05+00:00,"I'm @mcdonalds (9831 Las Tunas Dr, Temple City...","{'username': 'cat1152114', 'id': 30341217, 'di...","ÜT: 34.129889,-118.069719",34.085286,-117.960897
840,2010-11-19 23:48:02+00:00,"I'm @mcdonalds (8548 Valley Blvd, Rosemead) h...","{'username': 'cat1152114', 'id': 30341217, 'di...","ÜT: 34.129889,-118.069719",34.085286,-117.960897


...

,date,content,user,user_location,lat,lng
0,2021-06-21 04:02:33+00:00,@McDonalds I meant to say McDonald's is good a...,"{'username': 'sirhappy0ne', 'id': 281752132, '...",798,36.330228,-119.292058
1,2021-05-07 05:10:59+00:00,Why does @McDonalds say the ice cream machine ...,"{'username': '_Navarro_64_', 'id': 3310780010,...","Visalia, CA 🥑Michoacan, MX",36.330228,-119.292058
2,2020-12-11 05:20:38+00:00,@McDonalds The McRib has been crap since the m...,"{'username': 'shawnfinch14', 'id': 433895998, ...","Farmersville, CA",36.330228,-119.292058
3,2020-09-25 23:51:25+00:00,"@jatinsub @McDonalds Shooters shoot, thought y...","{'username': 'iAMJohnson13', 'id': 564190933, ...",,36.330228,-119.292058
4,2020-09-25 23:43:34+00:00,Cactus Jack sent me to @McDonalds #CactusJackS...,"{'username': 'iAMJohnson13', 'id': 564190933, ...",,36.330228,-119.292058
5,2020-01-30 23:24:53+00:00,@vkerce @McDonalds I’m done eating meat,"{'username': 'cservin8705', 'id': 483127577, '...","Visalia, CA",36.330228,-119.292058
6,2020-01-30 23:17:17+00:00,@vkerce @McDonalds Dude my heart hurts. I need...,"{'username': 'cservin8705', 'id': 483127577, '...","Visalia, CA",36.330228,-119.292058
7,2019-10-22 16:51:21+00:00,Will never buy a @McDonalds iced coffee again....,"{'username': 'CAL9487', 'id': 34317187, 'displ...","Visalia, CA",36.330228,-119.292058
8,2019-10-09 02:25:55+00:00,@JimVerros @McDonalds Well @McDonalds couldn’t...,"{'username': 'deebotz', 'id': 42345534, 'displ...","Fresno, CA",36.330228,-119.292058
9,2019-10-06 05:05:16+00:00,@McDonalds What the heck is this??? How do yo...,"{'username': 'laceechambs', 'id': 139328601, '...",,36.330228,-119.292058


,date,content,user,user_location,lat,lng
0,2021-07-03 13:39:35+00:00,@ChasingMemes0 @McDonalds @chasingscratch0 @ve...,"{'username': 'BrysonsNft', 'id': 1374194894089...",,33.200036,-117.242536
1,2021-07-03 13:28:16+00:00,@ChasingMemes0 @McDonalds @chasingscratch0 @ve...,"{'username': 'BrysonsNft', 'id': 1374194894089...",,33.200036,-117.242536
2,2021-06-07 22:56:47+00:00,no way🥰😵 @McDonalds https://t.co/phhGmGZJ0l,"{'username': 'hotdoghottie96', 'id': 68737379,...",,33.200036,-117.242536
3,2021-06-02 03:41:44+00:00,I’ve eaten so many french fries this week😴 @Mc...,"{'username': 'hotdoghottie96', 'id': 68737379,...",,33.200036,-117.242536
4,2021-05-31 04:17:02+00:00,@Still_HB @McDonalds Lol. Exactly,"{'username': 'jimb16149', 'id': 2894039977, 'd...","Slam Diego, CA",33.200036,-117.242536
...,...,...,...,...,...,...
297,2012-07-31 02:17:47+00:00,@McDonalds Please follow me please,"{'username': 'wwe619er', 'id': 325106944, 'dis...",san diego,33.200036,-117.242536
298,2012-07-25 05:50:13+00:00,"I'm @mcdonalds (701 Center Dr, San Marcos) ht...","{'username': 'JamesonMahiai', 'id': 366504883,...","San Diego, CA",33.200036,-117.242536
299,2012-07-19 07:22:39+00:00,@mcdonalds http://t.co/bdjfwLU (+5pts),"{'username': 'JamesonMahiai', 'id': 366504883,...","San Diego, CA",33.200036,-117.242536
300,2012-07-08 08:45:25+00:00,"I'm @mcdonalds (701 Center Dr, San Marcos) ht...","{'username': 'JamesonMahiai', 'id': 366504883,...","San Diego, CA",33.200036,-117.242536


,date,content,user,user_location,lat,lng
0,2021-07-08 01:31:58+00:00,@McDonalds I'll download your app after you do...,"{'username': 'Gilbert76679457', 'id': 90954154...","La Puente, CA",34.068622,-117.938953
1,2021-07-07 05:02:40+00:00,@McDonalds bring back strawberry sundaes and p...,"{'username': 'anadancee1', 'id': 3224880523, '...","Diamond Bar, CA",34.068622,-117.938953
2,2021-07-06 07:04:21+00:00,@McDonalds I see something I would never put i...,"{'username': 'NicoleSciarra', 'id': 40206475, ...",CaLiFoRnIa/OrEgOn 626/541,34.068622,-117.938953
3,2021-07-02 07:03:45+00:00,I’ve been seeing a lot of people talk about ho...,"{'username': 'admayral_me', 'id': 474295148, '...",,34.068622,-117.938953
4,2021-05-27 13:39:53+00:00,"@McDonalds Yes. Yes, I am. Don't care.","{'username': 'Jenn8590', 'id': 1486275330, 'di...",Californ -I- A,34.068622,-117.938953
...,...,...,...,...,...,...
773,2011-01-11 01:20:19+00:00,"I'm @mcdonalds (7503 Garvey Ave, Rosemead) ht...","{'username': 'cat1152114', 'id': 30341217, 'di...","ÜT: 34.129889,-118.069719",34.068622,-117.938953
774,2011-01-05 17:10:27+00:00,"I'm @mcdonalds (8548 Valley Blvd, Rosemead) h...","{'username': 'cat1152114', 'id': 30341217, 'di...","ÜT: 34.129889,-118.069719",34.068622,-117.938953
775,2010-12-10 17:13:58+00:00,"I'm @mcdonalds (7503 Garvey Ave, Rosemead) ht...","{'username': 'cat1152114', 'id': 30341217, 'di...","ÜT: 34.129889,-118.069719",34.068622,-117.938953
776,2010-11-21 03:39:05+00:00,"I'm @mcdonalds (9831 Las Tunas Dr, Temple City...","{'username': 'cat1152114', 'id': 30341217, 'di...","ÜT: 34.129889,-118.069719",34.068622,-117.938953


,date,content,user,user_location,lat,lng
0,2021-07-13 13:37:26+00:00,@KrisAndersonTV @BurgerKing @McDonalds @Chickf...,"{'username': 'NoSpunZone', 'id': 1221948777184...","Seal Beach, CA",33.751342,-117.993992
1,2021-07-12 16:27:33+00:00,@McDonalds A fired dead chicken,"{'username': 'SunNightMoonSt1', 'id': 11151024...","Santa Ana, CA",33.751342,-117.993992
2,2021-07-08 15:13:46+00:00,"@McDonalds 55,000 (year McDs opened 10000x)","{'username': 'hastin', 'id': 15002733, 'displa...","Santa Ana, CA",33.751342,-117.993992
3,2021-07-04 04:23:57+00:00,@digital_phreak @Tool @McDonalds Went at like ...,"{'username': 'Kayz42', 'id': 89940646270269030...","Huntington Beach, CA",33.751342,-117.993992
4,2021-07-04 01:48:50+00:00,"I'm at @McDonalds in Garden Grove, CA https://...","{'username': 'john143choco', 'id': 277850760, ...","Santa Ana, CA",33.751342,-117.993992
...,...,...,...,...,...,...
411,2017-01-31 23:06:48+00:00,@trvo512 @geneva_genevaaa @McDonalds that is f...,"{'username': 'PrimoHistory', 'id': 41724235, '...",Mission Viejo,33.751342,-117.993992
412,2017-01-22 20:10:27+00:00,I saw a commercial that mcpick 2 was back but ...,"{'username': 'Re_BirthofVenus', 'id': 47236084...",Antelope Valley,33.751342,-117.993992
413,2016-12-31 14:25:46+00:00,My chicken nuggets have betrayed me. I've thro...,"{'username': 'tiffanyaxe1', 'id': 231980126, '...",,33.751342,-117.993992
414,2016-12-09 18:42:28+00:00,@McDonalds hashbrowns are the way to my heart,"{'username': '_emileeh_', 'id': 741482102, 'di...","California, USA",33.751342,-117.993992


,date,content,user,user_location,lat,lng
0,2021-07-12 05:38:52+00:00,Join us tomorrow to kick-off @McDonalds pop-up...,"{'username': 'CGarcia_CA', 'id': 7460166300068...","Bell Gardens, CA",33.979178,-118.032844
1,2021-07-11 07:40:41+00:00,@DeonnaPurrazzo @McDonalds Have u ever read sc...,"{'username': 'JackBlazzer', 'id': 128528040789...","Bell Gardens, CA",33.979178,-118.032844
2,2021-07-08 01:31:58+00:00,@McDonalds I'll download your app after you do...,"{'username': 'Gilbert76679457', 'id': 90954154...","La Puente, CA",33.979178,-118.032844
3,2021-07-07 23:16:54+00:00,@Dogecoinxx @McDonalds Make it so !!,"{'username': 'rigobear72', 'id': 26048598, 'di...","Norwalk, CA",33.979178,-118.032844
4,2021-07-07 21:51:22+00:00,@McDonalds the view on Independence Day https:...,"{'username': 'Sol54012217', 'id': 106044863992...",,33.979178,-118.032844
5,2021-07-06 23:36:05+00:00,"I'm at @McDonalds in Downey, CA https://t.co/1...","{'username': 'ColbyLindsay', 'id': 26492977, '...",Atlanta,33.979178,-118.032844


In [50]:
print(loc)

37.765206, -122.241636, 10km


In [51]:
df_test = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    '@McDonalds geocode:"{}"'.format(loc)).get_items(), 1000))[['date', 'content']]

In [97]:
final_df_array = []

for df in df_array:
    print(df['date'].count())
    if df['date'].count() > 30:
        final_df_array.append(df)
        


115
1192
46
218
842
8
15
372
13
239
966
46
20
49
394
16
48
83
504
1422
44
245
24
173
71
50
32
392
1153
1233
44
31
10
18
35
75
146
4606
344
4
54
64
100
138
74
46
162
3
37
363
8
91
214
423
506
204
38
10
237
370
22
358
136
76
93
114
306
69
339
67
1255
1730
370
231
18
681
119
1783
229
75
3756
70
16
16
186
125
151
6
658
109
485
8
31
18
68
59
302
778
416
6


In [110]:
for df in final_df_array:
    print(df['date'].count())

115
1192
46
218
842
372
239
966
46
49
394
48
83
504
1422
44
245
173
71
50
32
392
1153
1233
44
31
35
75
146
4606
344
54
64
100
138
74
46
162
37
363
91
214
423
506
204
38
237
370
358
136
76
93
114
306
69
339
67
1255
1730
370
231
681
119
1783
229
75
3756
70
186
125
151
658
109
485
31
68
59
302
778
416


In [119]:
rows = []

for df in final_df_array:
    sentiment_total = 0
    df_len = df['date'].count()
    i = 0
    while i < df_len:
        testimonial = TextBlob(df.loc[i, 'content'])
        sentiment_total = sentiment_total + testimonial.sentiment.polarity
        sentiment_average = sentiment_total/df_len
        i = i+1
    print(f"({df.loc[1,'lat']}, {df.loc[1,'lng']}): Sentiment Total = {sentiment_total}, Average Sentiment: {sentiment_average}")
    rows.append([df.loc[1,'lat'],df.loc[1,'lng'], sentiment_total, sentiment_average])
    print("-" *42)

(34.095286, -118.127014): Sentiment Total = 5.967458400974029, Average Sentiment: 0.05189094261716547
------------------------------------------
(33.835292, -117.914503): Sentiment Total = 49.27553906328462, Average Sentiment: 0.041338539482621325
------------------------------------------
(38.004922, -121.805789): Sentiment Total = 2.962361111111111, Average Sentiment: 0.06439915458937198
------------------------------------------
(35.373292, -119.018711): Sentiment Total = 8.748301767676768, Average Sentiment: 0.0401298246223705
------------------------------------------
(34.085286, -117.960897): Sentiment Total = 21.640487877284762, Average Sentiment: 0.025701292015777626
------------------------------------------
(33.867514, -117.998117): Sentiment Total = 22.904552569826002, Average Sentiment: 0.06157137787587635
------------------------------------------
(33.158092, -117.350594): Sentiment Total = 6.848871339886965, Average Sentiment: 0.028656365438857592
------------------------

In [120]:
total_sentiment_df = pd.DataFrame(rows, columns=["lat", "lng", "sentiment_total", "sentiment_average"])

In [121]:
total_sentiment_df

,lat,lng,sentiment_total,sentiment_average
0,34.095286,-118.127014,5.967458,0.051891
1,33.835292,-117.914503,49.275539,0.041339
2,38.004922,-121.805789,2.962361,0.064399
3,35.373292,-119.018711,8.748302,0.040130
4,34.085286,-117.960897,21.640488,0.025701
...,...,...,...,...
75,34.536108,-117.291158,-1.057666,-0.015554
76,36.330228,-119.292058,2.263889,0.038371
77,33.200036,-117.242536,18.123441,0.060011
78,34.068622,-117.938953,21.133219,0.027164


In [124]:
total_sentiment_df.to_csv("mcdonalds_tweets_average_sentiments.csv")


In [126]:
# Specify the file to write to
output_path = os.path.join("..","tweet_csvs", "mcdonalds_tweets.csv")

# Open the file using "write" mode. Specify the variable to hold the contents
with open(output_path, 'w', newline='') as csvfile:
    
    # Initialize csv.writer
    csvwriter = csv.writer(csvfile, delimiter=',')
    
    csvwriter.writerow(['date', 'content', 'lat', 'lng', 'city', 'population', 'sentiment'])
    
    for df in final_df_array:
        print(df['date'].count())
        everything_bigger_final = pd.merge(df, final_city_df, how='inner', left_on=['lat', 'lng'], right_on=['lat', 'lng'])
        
        print(everything_bigger_final['date'].count())
        
        index_len = everything_bigger_final['date'].count()
        i = 0
        while i < index_len:
            testimonial = TextBlob(everything_bigger_final.loc[i, 'content'])
            
            csvwriter.writerow([everything_bigger_final.loc[i, 'date'], everything_bigger_final.loc[i, 'content'], everything_bigger_final.loc[i, 'lat'], everything_bigger_final.loc[i,'lng'], everything_bigger_final['city'][0], everything_bigger_final['pop_april_2010'][0], testimonial.sentiment.polarity])
            i = i+1
    # Write the first row (column headers)

    #date	content	user	user_location	lat	lng

115
115
1192
1192
46
46
218
218
842
842
372
372
239
239
966
966
46
46
49
49
394
394
48
48
83
83
504
504
1422
1422
44
44
245
245
173
173
71
71
50
50
32
32
392
392
1153
1153
1233
1233
44
44
31
31
35
35
75
75
146
146
4606
4606
344
344
54
54
64
64
100
100
138
138
74
74
46
46
162
162
37
37
363
363
91
91
214
214
423
423
506
506
204
204
38
38
237
237
370
370
358
358
136
136
76
76
93
93
114
114
306
306
69
69
339
339
67
67
1255
1255
1730
1730
370
370
231
231
681
681
119
119
1783
1783
229
229
75
75
3756
3756
70
70
186
186
125
125
151
151
658
658
109
109
485
485
31
31
68
68
59
59
302
302
778
778
416
416
